In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression as linreg
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn import linear_model
import math

df = pd.read_csv("fifa21_train.csv")

In [6]:
def preprocess(df):
    df = df.drop_duplicates(keep = False)
    df['bp_info'] = df.apply(lambda row: row[row['BP']], axis=1)
    df['bp_info'] = df.apply(lambda row: row[row['BP']] if row['BP'] in row.index else None, axis=1)
    df.columns = df.columns.str.lower().str.replace(' ','_')
    df.columns = df.columns.str.strip()
    df['bp_info'] = df['bp_info'].apply(lambda x : x.split('+')[0])
    df['bp_info'] = df['bp_info'].astype('int64')
    columns_needed = ['age', 'bp_info', 'wage', 'value', 'attacking', 'skill', 'movement', 'power', 'mentality', 'defending', 'goalkeeping', 'total_stats', 'base_stats', 'ir', 'ova']
    df = df.loc[:, columns_needed]
    pd.to_numeric(df['ir'].str.rstrip('★'), errors='coerce')
    df['ir'] = pd.to_numeric(df['ir'].str.rstrip('★'), errors='coerce')
    df['wage'] = df['wage'].str.replace('€', '').str.replace('K', '000').str.replace('M', '000000').str.replace('.','')
    df['value'] = df['value'].str.replace('€', '').str.replace('K', '000').str.replace('M', '000000').str.replace('.','')
    df['bp_info'] = df['bp_info'].astype(str).str.replace('€', '').str.replace('K', '000').str.replace('M', '000000').str.replace('.','').str.replace(r'\+\d+', '', regex=True)
    df['wage'] = pd.to_numeric(df['wage'], errors='coerce')
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df['bp_info'] = df['bp_info'].apply(lambda x : x.split('+')[0])
    df['bp_info'] = df['bp_info'].astype('int64')
    return df
df = preprocess(df)
df

,age,bp_info,wage,value,attacking,skill,movement,power,mentality,defending,goalkeeping,total_stats,base_stats,ir,ova
0,26,63,4000,525000,258,286,346,306,290,148,48,1682,357,1,64
1,30,77,23000,85000000,365,375,404,372,313,77,55,1961,412,2,77
2,33,80,49000,9000000,336,424,424,308,332,80,21,1925,404,2,80
3,22,58,4000,275000,242,259,282,277,257,168,42,1527,329,1,59
4,23,64,2000,725000,249,280,324,280,294,185,52,1664,360,1,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,25,63,3000,625000,224,248,333,273,266,184,61,1589,347,1,65
11697,27,69,4000,16000000,84,96,237,217,101,41,347,1123,387,1,70
11698,22,66,2000,11000000,286,291,388,278,241,62,57,1603,337,1,67
11699,26,71,0,0,254,284,198,258,241,113,81,1429,400,3,68


In [7]:
y = df['ova']
x = df.drop(['ova'], axis=1)

In [8]:
import pickle
df = pd.read_csv("fifa21_train.csv")
names = ['age', 'bp_info', 'wage', 'value', 'attacking', 'skill', 
         'movement', 'power', 'mentality', 'defending', 'goalkeeping', 'total_stats', 'base_stats', 'ir', 'ova']
array = df.values
X = array[:,0:8]
Y = array[:,8]
test_size = 0.33
seed = 7
x_train, x_test, y_train, y_test = tts(x, y, test_size=test_size, random_state=seed)
# Fit the model on training set
lm = linear_model.LinearRegression()
model = lm.fit(x_train, y_train)
# save the model to disk
filename = 'final_fifa_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)

0.9804144817733295
